In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)

In [2]:
mol_str = """
0 1
#O
#H 1 1.1
#H 1 1.1 2 104
#n
#n 1 1.0
#li
#h 1 1.0
o
h 1 1.0
h 1 1.0 2 104.0
#be
#h 1 1.0
#h 1 1.2 2 104.0
symmetry c1
"""

options_dict = {'basis': '6-31g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }

mol = psi4.geometry(mol_str)


psi4.set_options(options_dict)
psi4.core.set_output_file('output.dat', False)

In [3]:
# photon energy
#omega_val = 2.0 / psi4.constants.Hartree_energy_in_eV
omega_val = 0

print(omega_val)

# lambda vector
lambda_vector = np.array([0., 0., 0.0])
#lambda_vector = np.array([0.5, 0.5, 0.5])
#number of active orbitals
n_act_orb=5
#number of active electrons
n_act_el=4

# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(1, mol_str, options_dict, lambda_vector,omega_val, n_act_el,n_act_orb,ignore_coupling=True, cas=False)
nmo=H2_PF.nmo
ndocc=H2_PF.ndocc
numDets=H2_PF.numDets
excitation_index=H2_PF.excitation_index
singdetsign=H2_PF.singdetsign
print(nmo)
print(ndocc)
print(numDets)
print(excitation_index)
# uncomment to print
#print(H2_PF.H_PF)


0

Start SCF iterations:

Canonical RHF One-electron energy = -122.2743570184695727
CQED-RHF One-electron energy      = -122.2743570184695727
Nuclear repulsion energy          = 8.8026031303907679
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -75.9800163449004344   dE = -7.59800E+01   dRMS = 5.74342E-13
SCF Iteration   2: Energy = -75.9800163449003350   dE =  9.94760E-14   dRMS = 1.52534E-13
Total time for SCF iterations: 0.001 seconds 

QED-RHF   energy: -75.98001634 hartree
Psi4  SCF energy: -75.98001634 hartree
[0, 1, 2, 3, 4]
betalist
[0, 1, 2, 3, 5]
[9, 11]
betalist
[0, 1, 2, 3, 6]
[9, 13]
betalist
[0, 1, 2, 3, 7]
[9, 15]
betalist
[0, 1, 2, 3, 8]
[9, 17]
betalist
[0, 1, 2, 3, 9]
[9, 19]
betalist
[0, 1, 2, 3, 10]
[9, 21]
betalist
[0, 1, 2, 3, 11]
[9, 23]
betalist
[0, 1, 2, 3, 12]
[9, 25]
betalist
[0, 1, 2, 4, 5]
[7, 11]
betalist
[0, 1, 2, 4, 6]
[7, 13]
betalist
[0, 1, 2, 4, 7]
[7, 15]
betalist
[0, 1, 2, 4, 8]
[7, 17]
betalist
[0, 1, 2, 4, 9]
[7,

In [4]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

#print the ground-state and first excited state
#print("Psi4 |G> is ",fci_e)
print("|G> is ",e_fci[0])
print("|LP> is ",e_fci[1])
nsingdet=numDets-1
np.savetxt('h_cis', H2_PF.H_PF[:nsingdet+1,:nsingdet+1])

for n in range(wavefunctions.shape[1]):
    print(n, e_fci[n])
c00 = wavefunctions[0,14]
c10 = wavefunctions[1:nsingdet+1,14]
c01 = wavefunctions[nsingdet+1,14]
c11 = wavefunctions[(nsingdet+2):,14]
#print(wavefunctions[:,5])
print(c10)
print(c11)
print(c11.shape)
print('aaaa')
#print(c11)
nvirt=nmo-ndocc
c10_large =np.zeros((4*ndocc*nvirt))
c11_large =np.zeros((4*ndocc*nvirt))
for n in range(nsingdet):
    ia = excitation_index[n]
    a = ia%(2*nvirt)
    i = (ia-a)//(2*nvirt)
    #print (i,a+2*ndocc,excitation_index[n])
    c10_large[excitation_index[n]]=c10[n]*singdetsign[n]
    c11_large[excitation_index[n]]=c11[n]*singdetsign[n]

    
for m in range(wavefunctions.shape[1]):
    print('state', m) #end = '')
    #print (wavefunction[1:singdet+1,m])
    for n in range(wavefunctions.shape[0]):        
        if n == 0:
            sing_coeff = wavefunctions[n,m]
            if np.abs(sing_coeff) >0.01:
                print(' zero excitation')
        if n>0 and n <= nsingdet:
            #print(' single exciation with 0 photon')
            sing_coeff = wavefunctions[n,m]
            if np.abs(sing_coeff) >0.01:
                ia = excitation_index[n-1]
                a = ia%(2*nvirt)
                i = (ia-a)//(2*nvirt)
                print (i, '->', a+2*ndocc, sing_coeff, 'zero photon')
        if n == nsingdet+1:
            sing_coeff = wavefunctions[n,m]
            if np.abs(sing_coeff) >0.01:
                print(' zero excitation with 1 photon')
        if n > nsingdet+1:
            #print(' single exciation with 1 photon')
            sing_coeff = wavefunctions[n,m]
            if np.abs(sing_coeff) >0.01:
                ia = excitation_index[n-nsingdet-2]
                a = ia%(2*nvirt)
                i = (ia-a)//(2*nvirt)
                print (i, '->', a+2*ndocc, sing_coeff, 'one photon')
    
    
    
    
    
    
print(c10_large)
Dij = np.zeros((2*ndocc,2*ndocc))
Dab = np.zeros((2*nvirt,2*nvirt))
for i in range(2*ndocc):
    for j in range(2*ndocc):
        Dij[i,j] = (c00 * c00 + c01 * c01) * (i==j)
        Dij[i,j] += (np.dot(c10.T,c10)+np.dot(c11.T,c11)) * (i==j)
        dum = 0.0
        for a in range(2*nvirt):
            ia = i * 2*nvirt + a
            ja = j * 2*nvirt + a
            dum += c10_large[ia] * c10_large[ja]
            dum += c11_large[ia] * c11_large[ja]
        Dij[i,j] -=dum    

#print(Dij)
print(np.trace(Dij))
for a in range(2*nvirt):
    for b in range(2*nvirt):
        dum = 0.0
        for i in range(2*ndocc):
            ia = i * 2*nvirt + a
            ib = i * 2*nvirt + b
            dum += c10_large[ia] * c10_large[ib]
            dum += c11_large[ia] * c11_large[ib]
        Dab[a,b] += dum    
print(np.trace(Dab))             
print(np.trace(Dij)+np.trace(Dab)) 
Dia = np.zeros((2*ndocc,2*nvirt))

for i in range(2*ndocc):
    for a in range(2*nvirt):
        ia = i * 2*nvirt + a
        Dia[i,a] = c10_large[ia] * c00
        Dia[i,a] +=c11_large[ia] * c01
print('Dia')
print(Dia)
print('Dij')
print(Dij)
D1=np.concatenate((Dij,Dia),axis=1)
print(D1)
D2=np.concatenate((Dia.T,Dab),axis=1)
D3=np.concatenate((D1,D2),axis=0)
h1=H2_PF.Hspin
h2=H2_PF.antiSym2eInt

e1_test = np.einsum("pq,pq->", h1, D3)
#for p in range(2*nmo):
#    for q in range(2*nmo):
print(e1_test)             
print(np.trace(Dab))             
print(np.trace(Dij)+np.trace(Dab))     

|G> is  -75.98001634490035
|LP> is  -75.98001634490035
0 -75.98001634490035
1 -75.98001634490035
2 -75.68871175368106
3 -75.68871175368102
4 -75.65207659122954
5 -75.65207659122949
6 -75.62788317789175
7 -75.62788317789172
8 -75.61052301274427
9 -75.61052301274417
10 -75.5851172306854
11 -75.5851172306854
12 -75.5687900731924
13 -75.56879007319239
14 -75.56001696513623
15 -75.56001696513623
16 -75.50292462969306
17 -75.50292462969296
18 -75.48971899099932
19 -75.4897189909993
20 -75.44852599511307
21 -75.44852599511304
22 -75.44021189935805
23 -75.44021189935802
24 -75.29777460510111
25 -75.2977746051011
26 -75.01182365815161
27 -75.01182365815158
28 -74.90816437990519
29 -74.90816437990505
30 -74.88576404745466
31 -74.88576404745461
32 -74.86821718113686
33 -74.86821718113686
34 -74.86235889790765
35 -74.8623588979076
36 -74.85749272166618
37 -74.85749272166616
38 -74.85648256782181
39 -74.85648256782177
40 -74.85448840637561
41 -74.85448840637552
42 -74.8336788585664
43 -74.833678858

In [5]:
tempp=np.einsum("pq,q->p", H2_PF.H_1E,wavefunctions[:,14])
e1_test2 = np.dot(wavefunctions[:,14].T,tempp)
print(e1_test2-H2_PF.Enuc)
print(e1_test-e1_test2+H2_PF.Enuc)

-119.48745625356835
3.552713678800501e-15


In [6]:
F =np.zeros((2*nmo,2*nmo))
for p in range(2*nmo):
    for q in range(2*nmo):
        F[p][q]= 1.0 * h1[p][q]
        for k in range(2*ndocc):
            F[p][q] += h2[p,k,q,k]
H11 =np.zeros((nsingdet,nsingdet))
E0 = H2_PF.cqed_reference_energy

for m in range(nsingdet):
    ia = excitation_index[m]
    a = ia%(2*nvirt)
    i = (ia-a)//(2*nvirt)
    for n in range(nsingdet):
        jb = excitation_index[n]
        b = jb%(2*nvirt)
        j = (jb-b)//(2*nvirt)
        H11[m][n]=E0*(m==n)+F[a+2*ndocc][b+2*ndocc]*(i==j)-F[i][j]*(a==b)+h2[a+2*ndocc,j,i,b+2*ndocc]
e_cis, cis_vec = np.linalg.eigh(H11)
print(e_cis)
#print(H11)
H11_large =np.zeros((4*ndocc*nvirt,4*ndocc*nvirt))
for i in range(2*ndocc):
    for a in range(2*nvirt):
        ia = i*2*nvirt+a
        for j in range(2*ndocc):
            for b in range(2*nvirt):
                jb = j*2*nvirt+b
                H11_large[ia][jb]=F[a+2*ndocc][b+2*ndocc]*(i==j)-F[i][j]*(a==b)+h2[a+2*ndocc,j,i,b+2*ndocc]
#e_cis2, cis_vec2 = np.linalg.eigh(H11_large)
#print(e_cis2) 
# H11_one =np.zeros((nsingdet,nsingdet))
# for m in range(nsingdet):
#     ia = excitation_index[m]
#     a = ia%(2*nvirt)
#     i = (ia-a)//(2*nvirt)
#     for n in range(nsingdet):
#         jb = excitation_index[n]
#         b = jb%(2*nvirt)
#         j = (jb-b)//(2*nvirt)
#         H11_one[m][n]=  h1[a+2*ndocc][b+2*ndocc]*(i==j)-h1[i][j]*(a==b)

# one_e_ext=np.einsum("pq,qr,rs->ps", cis_vec.T,H11_one,cis_vec)
# print(np.diagonal(one_e_ext)+H2_PF.cqed_one_energy)
H11_two =np.zeros((nsingdet+1,nsingdet+1))
H12 =np.zeros((nsingdet+1,nsingdet+1))

h1=H2_PF.Hspin
h2=H2_PF.antiSym2eInt
for m in range(nsingdet+1):
    for n in range(nsingdet+1):

        if m == 0 and n == 0:
            H11_two[m][n]= E0
        elif m == 0 and n > 0:
            jb = excitation_index[n-1]
            b = jb%(2*nvirt)
            j = (jb-b)//(2*nvirt)
            H11_two[m][n]= F[j][b+2*ndocc]
        elif m > 0 and n == 0:
            ia = excitation_index[m-1]
            a = ia%(2*nvirt)
            i = (ia-a)//(2*nvirt)
            H11_two[m][n]= F[i][a+2*ndocc]
        else:    
            ia = excitation_index[m-1]
            a = ia%(2*nvirt)
            i = (ia-a)//(2*nvirt)
            jb = excitation_index[n-1]
            b = jb%(2*nvirt)
            j = (jb-b)//(2*nvirt)
            H11_two[m][n]=E0*(m==n)+F[a+2*ndocc][b+2*ndocc]*(i==j)-F[i][j]*(a==b)+h2[a+2*ndocc,j,i,b+2*ndocc]
#e_cis2, cis_vec2 = np.linalg.eigh(H11_two)
#for i in range(cis_vec2.shape[1]):
#    print(e_cis2[i])
np.savetxt('h_cis2',H11_two)
    
    
#for i in range(cis_vec.shape[1]):
#    print(e_cis2[i+1]-e_cis[i])
    
Ht=np.concatenate((H11_two,H12),axis=1)

Hb=np.concatenate((H12,H11_two),axis=1)
Htt=np.concatenate((Ht,Hb),axis=0)
e_cis3, cis_vec3 = np.linalg.eigh(Htt)
for i in range(cis_vec3.shape[1]):
    print(e_cis3[i]-e_fci[i])

print(Htt-H2_PF.H_PF)
np.allclose(np.abs(Htt),np.abs(H2_PF.H_PF),1e-12,1e-12)


H11_one =np.zeros((nsingdet+1,nsingdet+1))
for m in range(nsingdet+1):
    for n in range(nsingdet+1):

        if m == 0 and n == 0:
            H11_one[m][n]= H2_PF.cqed_one_energy
        elif m == 0 and n > 0:
            jb = excitation_index[n-1]
            b = jb%(2*nvirt)
            j = (jb-b)//(2*nvirt)
            H11_one[m][n]= h1[j][b+2*ndocc]
        elif m > 0 and n == 0:
            ia = excitation_index[m-1]
            a = ia%(2*nvirt)
            i = (ia-a)//(2*nvirt)
            H11_one[m][n]= h1[i][a+2*ndocc]
        else:    
            ia = excitation_index[m-1]
            a = ia%(2*nvirt)
            i = (ia-a)//(2*nvirt)
            jb = excitation_index[n-1]
            b = jb%(2*nvirt)
            j = (jb-b)//(2*nvirt)
            H11_one[m][n]=H2_PF.cqed_one_energy*(m==n)+h1[a+2*ndocc][b+2*ndocc]*(i==j)-h1[i][j]*(a==b)
            
np.savetxt('h_one',H11_one)            
H1t=np.concatenate((H11_one,H12),axis=1)

H1b=np.concatenate((H12,H11_one),axis=1)
H1tt=np.concatenate((H1t,H1b),axis=0)
one_e_ext=np.einsum("pq,qr,rs->ps", cis_vec3.T,H1tt,cis_vec3)
print(np.diagonal(one_e_ext))
one_e_ext2=np.einsum("pq,qr,rs->ps", cis_vec3.T,H2_PF.H_1E,cis_vec3)
print(np.diagonal(one_e_ext2)-H2_PF.Enuc)

np.allclose(np.abs(H1tt),np.abs(H2_PF.H_1E-H2_PF.Enuc*np.identity(H1tt.shape[0])),1e-12,1e-12)


[-75.68871175 -75.65207659 -75.62788318 -75.61052301 -75.58511723
 -75.56879007 -75.56001697 -75.50292463 -75.48971899 -75.448526
 -75.4402119  -75.29777461 -75.01182366 -74.90816438 -74.88576405
 -74.86821718 -74.8623589  -74.85749272 -74.85648257 -74.85448841
 -74.83367886 -74.82930638 -74.80550348 -74.80103277 -74.79843697
 -74.79583451 -74.76511969 -74.75274383 -74.72758682 -74.72041406
 -74.70255612 -74.69857449 -74.68754565 -74.67670914 -74.66285801
 -74.65117589 -74.63564651 -74.62122072 -74.61839601 -74.60948906
 -74.59052349 -74.57044164 -74.53998309 -74.48862477 -74.48651193
 -74.37912791 -74.27286631 -74.26729752 -74.2132889  -74.14461061
 -74.13642741 -74.07894194 -74.06384642 -74.0631734  -74.06080677
 -74.02000119 -74.01076979 -73.96675531 -73.93299218 -73.86644618
 -73.83964325 -73.79761068 -73.52092816 -73.35776147 -55.81703606
 -55.79363636 -55.74977938 -55.72828207 -55.22461868 -55.15886593
 -55.13082726 -55.07908524 -54.92820487 -54.88342497 -54.83087575
 -54.8105738

[-122.27435702 -122.27435702 -119.44761764 -119.44761764 -119.33317263
 -119.33317263 -119.74318883 -119.74318883 -119.66142657 -119.66142657
 -119.57125063 -119.57125063 -119.97395619 -119.97395619 -119.48745625
 -119.48745625 -119.89968021 -119.89968021 -119.74182303 -119.74182303
 -120.16625526 -120.16625526 -119.70776685 -119.70776685 -119.65721109
 -119.65721109 -120.23214484 -120.23214484 -120.35212543 -120.35212543
 -119.90213915 -119.90213915 -118.54675415 -118.54675415 -120.25310112
 -120.25310112 -119.5897241  -119.5897241  -118.58204362 -118.58204362
 -119.00614771 -119.00614771 -120.32784475 -120.32784475 -120.10753981
 -120.10753981 -118.99302378 -118.99302378 -119.155256   -119.155256
 -120.69939565 -120.69939565 -119.12296066 -119.12296066 -119.18792249
 -119.18792249 -118.90237583 -118.90237583 -119.00187992 -119.00187992
 -118.78509878 -118.78509878 -120.06651561 -120.06651561 -119.82114674
 -119.82114674 -119.11373528 -119.11373528 -119.1849344  -119.1849344
 -119.918

True

In [7]:
c00n = cis_vec3[0,14]
c10n = cis_vec3[1:nsingdet+1,14]
c01n = cis_vec3[nsingdet+1,14]
c11n = cis_vec3[(nsingdet+2):,14]
#print(c11)
nvirt=nmo-ndocc
c10n_large =np.zeros((4*ndocc*nvirt))
c11n_large =np.zeros((4*ndocc*nvirt))
for n in range(nsingdet):
    ia = excitation_index[n]
    a = ia%(2*nvirt)
    i = (ia-a)//(2*nvirt)
    #print (i,a+2*ndocc,excitation_index[n])
    c10n_large[excitation_index[n]]=c10n[n]
    c11n_large[excitation_index[n]]=c11n[n]

    
# for m in range(cis_vec3.shape[1]):
#     print('state', m) #end = '')
#     #print (wavefunction[1:singdet+1,m])
#     for n in range(cis_vec3.shape[0]):        
#         if n == 0:
#             sing_coeff = cis_vec3[n,m]
#             if np.abs(sing_coeff) >0.01:
#                 print(' zero excitation')
#         if n>0 and n <= nsingdet:
#             #print(' single exciation with 0 photon')
#             sing_coeff = cis_vec3[n,m]
#             if np.abs(sing_coeff) >0.01:
#                 ia = excitation_index[n-1]
#                 a = ia%(2*nvirt)
#                 i = (ia-a)//(2*nvirt)
#                 print (i, '->', a+2*ndocc, sing_coeff, 'zero photon')
#         if n == nsingdet+1:
#             sing_coeff = cis_vec3[n,m]
#             if np.abs(sing_coeff) >0.01:
#                 print(' zero excitation with 1 photon')
#         if n > nsingdet+1:
#             #print(' single exciation with 1 photon')
#             sing_coeff = cis_vec3[n,m]
#             if np.abs(sing_coeff) >0.01:
#                 ia = excitation_index[n-nsingdet-2]
#                 a = ia%(2*nvirt)
#                 i = (ia-a)//(2*nvirt)
#                 print (i, '->', a+2*ndocc, sing_coeff, 'one photon')
    
    
    
    
    
    
print(c10n_large)
Dijn = np.zeros((2*ndocc,2*ndocc))
Dabn = np.zeros((2*nvirt,2*nvirt))
for i in range(2*ndocc):
    for j in range(2*ndocc):
        Dijn[i,j] = (c00n * c00n + c01n * c01n) * (i==j)
        Dijn[i,j] += (np.dot(c10n.T,c10n)+np.dot(c11n.T,c11n)) * (i==j)
        dum = 0.0
        for a in range(2*nvirt):
            ia = i * 2*nvirt + a
            ja = j * 2*nvirt + a
            dum += c10n_large[ia] * c10n_large[ja]
            dum += c11n_large[ia] * c11n_large[ja]
        Dijn[i,j] -=dum    

#print(Dij)
print(np.trace(Dijn))
for a in range(2*nvirt):
    for b in range(2*nvirt):
        dum = 0.0
        for i in range(2*ndocc):
            ia = i * 2*nvirt + a
            ib = i * 2*nvirt + b
            dum += c10n_large[ia] * c10n_large[ib]
            dum += c11n_large[ia] * c11n_large[ib]
        Dabn[a,b] += dum    
print(np.trace(Dabn))             
print(np.trace(Dijn)+np.trace(Dabn)) 
Dian = np.zeros((2*ndocc,2*nvirt))

for i in range(2*ndocc):
    for a in range(2*nvirt):
        ia = i * 2*nvirt + a
        Dian[i,a] = c10n_large[ia] * c00n
        Dian[i,a] +=c11n_large[ia] * c01n
print('Dia')
print(Dian)
print('Dij')
print(Dijn)
D1n=np.concatenate((Dijn,Dian),axis=1)
print(D1)
D2n=np.concatenate((Dian.T,Dabn),axis=1)
D3n=np.concatenate((D1n,D2n),axis=0)

e1_testn = np.einsum("pq,pq->", h1, D3n)
#for p in range(2*nmo):
#    for q in range(2*nmo):
print(e1_testn)             
 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
8.999999999999982
0.9999999999999981
9.99999999999998
Dia
[[ 2.96820982e-16  0.00000000e+00  1.74637731e-28  0.00000000e+00
   4.24098037e-29  0.00000000e+00 -2.52699505e-16  0.00000000e+00
   9.01395358e-29  0.00000000e+00 -6.11386015e-17  0.00000000e+00
  -7.99651729e-29  0.00000000e+00 -2.50953374e-16  0.00000000e+00]
 [ 0.00000000e+00  2.96820982e-16  0.00000000e+00  1.50515181e-28
   0.00000000e+00  3.93563865e-29  0.00000000e+00 -2.52699505e-16
   0.00000000e+00  1.43072823e-28  0.00000000e+00 -6.1138

In [8]:

# First compute SCF energy using Psi4
scf_e, wfn = psi4.energy('SCF', return_wfn=True)

# now compute cqed-rhf to get transformation vectors with cavity
cqed_rhf_dict = cqed_rhf(lambda_vector, mol_str, options_dict)
rhf_reference_energy = cqed_rhf_dict["RHF ENERGY"]
cqed_reference_energy = cqed_rhf_dict["CQED-RHF ENERGY"]
C = cqed_rhf_dict["CQED-RHF C"]



# collect rhf wfn object as dictionary
wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# update wfn_dict with orbitals from CQED-RHF
wfn_dict["matrix"]["Ca"] = C
wfn_dict["matrix"]["Cb"] = C
# update wfn object
wfn = psi4.core.Wavefunction.from_file(wfn_dict)
#psi4.set_options({'restricted_docc': [0],'active': [5]})
#fci_energy = psi4.energy('fci',ref_wfn=wfn)


Start SCF iterations:

Canonical RHF One-electron energy = -122.2743570184695727
CQED-RHF One-electron energy      = -122.2743570184695727
Nuclear repulsion energy          = 8.8026031303907679
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -75.9800163449004344   dE = -7.59800E+01   dRMS = 5.74342E-13
SCF Iteration   2: Energy = -75.9800163449003350   dE =  9.94760E-14   dRMS = 1.52534E-13
Total time for SCF iterations: 0.001 seconds 

QED-RHF   energy: -75.98001634 hartree
Psi4  SCF energy: -75.98001634 hartree


ndocc = 5
n_act_el = 4
n_act_orb = 4

n_in_orb=ndocc-n_act_el//2
n_ac_el_half=n_act_el//2
print(n_ac_el_half)
inactive_list=list(x for x in range(n_in_orb))
print(inactive_list)
print('Generating all determinants in active space')
t = time.time()
singlesDets = []
for alpha in combinations(range(n_act_orb), n_ac_el_half):
    alphalist=list(alpha)
    alphalist=[x+n_in_orb for x in alphalist]
    alphalist[0:0]=inactive_list
    alpha=tuple(alphalist)
    #alpha_ex_level = compute_excitation_level(alpha, ndocc)
    for beta in combinations(range(n_act_orb), n_ac_el_half):
        betalist=list(beta)
        betalist=[x+n_in_orb for x in betalist]
        betalist[0:0]=inactive_list
        beta=tuple(betalist)
        #beta_ex_level = compute_excitation_level(beta, ndocc)
        #if alpha_ex_level + beta_ex_level == 1:
        print(F' adding alpha: {alpha} and beta: {beta}\n') 
        singlesDets.append(Determinant(alphaObtList=alpha, betaObtList=beta))